In [1]:
import tensorflow as tf
import numpy as np

In [2]:
dataset = tf.data.TextLineDataset('file.txt')

In [ ]:
for line in dataset:
    print(line)

In [3]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [4]:
with tf.Session() as sess:
    for i in range(3):
        print(sess.run(next_element))

b'I use Tensorflow'
b'You use PyTorch'
b'Both are great'


In [5]:
dataset = dataset.map(lambda string: tf.string_split([string]).values)

In [6]:
dataset = dataset.shuffle(buffer_size=3)

In [7]:
dataset.batch(2)

<BatchDataset shapes: (?, ?), types: tf.string>

In [8]:
dataset.prefetch(1)

<PrefetchDataset shapes: (?,), types: tf.string>

In [9]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [10]:
with tf.Session() as sess:
    print(sess.run(next_element))

[b'You' b'use' b'PyTorch']


# use initalizer op.

In [13]:
dataset = tf.data.TextLineDataset("file.txt")
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
init_op = iterator.initializer

In [14]:
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(next_element))
    print(sess.run(next_element))
    sess.run(init_op)
    print(sess.run(next_element))

b'I use Tensorflow'
b'You use PyTorch'
b'I use Tensorflow'


## what I need to read from the dataset.
(1950, 512) ->
each 512 line is an input
convert each 512 line into [0,...num_sequences], [i+1, ... num_sequences] => total no. of sequences should be equal to 512. - Task 1

In [35]:
freq = [i for i in range(512)]
freq = np.array(freq)

In [51]:
# freq = np.random.sample((512, 1))
# labels = np.random.randint(0, 4, (512, 1))

In [36]:
freq = freq.reshape(freq.shape[0], 1)

In [37]:
batch_size = 32
num_steps = 16

In [38]:
frequencies = np.zeros((512, 16), dtype=np.float32)
print(frequencies.shape)

(512, 16)


In [42]:
frequencies[1].shape

(16,)

In [102]:
labels = np.random.randint(0, 4, (512, 1))

In [103]:
labels.shape

(512, 1)

In [59]:
# convert into num step sized repeated samples.
def convertStepSizes(freq, num_steps):
    for i in range(freq.shape[0]):
        if freq[i:i+num_steps, 0].shape[0] == num_steps:
            frequencies[i, :] = freq[i:i+num_steps, 0]
        else:
#             print(freq[i:i+num_steps, 0].shape[0])
            frequencies[i, :] = np.pad(freq[i:i+num_steps, 0], (0, 16-freq[i:i+num_steps, 0].shape[0]), 'edge')
    print('loading complete')

In [60]:
freq = convertStepSizes(freq, 16)

15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
loading complete


In [61]:
print(frequencies.shape)

(512, 16)


In [62]:
frequencies[0]

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15.], dtype=float32)

In [63]:
frequencies[1]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16.], dtype=float32)

In [64]:
frequencies[-1]

array([511., 511., 511., 511., 511., 511., 511., 511., 511., 511., 511.,
       511., 511., 511., 511., 511.], dtype=float32)

In [65]:
frequencies[0].shape

(16,)

In [66]:
frequencies.shape

(512, 16)

## Use the alternate method described in tensorflow - importing data tutorial.

In [69]:
frequencies.dtype

dtype('float32')

In [104]:
freq_placeholder = tf.placeholder(frequencies.dtype, frequencies.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

In [105]:
dataset = tf.data.Dataset.from_tensor_slices((freq_placeholder, labels_placeholder))

In [106]:
dataset = dataset.batch(batch_size)

In [107]:
iterator = dataset.make_initializable_iterator()
inputs, output_labels = iterator.get_next()

In [108]:
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={freq_placeholder: frequencies, labels_placeholder: labels})
    for i in range(3):
        print(sess.run(inputs))

[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
 [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
 [ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]
 [ 3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.]
 [ 4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20.]
 [ 6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21.]
 [ 7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22.]
 [ 8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23.]
 [ 9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25.]
 [11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26.]
 [12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27.]
 [13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28.]
 [14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 

## model parameters constants.
<font color='red'>Change parameters here for model tuning.</font>

In [96]:
hidden_layers = 32
number_layers = 2
dropout = 0.0

## Model initial state.

In [95]:
init_state = tf.placeholder(tf.float32, [number_layers, 2, batch_size, hidden_layers])
state_per_layer_list = tf.unstack(init_state, axis=0)
rnn_state_tuples = tuple([tf.nn.rnn_cell.LSTMStateTuple(state[0], state[1]) for state in state_per_layer_list])

## Model

In [ ]:
cell = tf.contrib.rnn.LSTMCell(hidden_size, forget_bias=1.0)
if dropout != '0.0':
    cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout)
    
if num_layers > 1:
    cell_list = [cell for _ in range(num_layers)]
    cell = tf.nn.rnn_cell.MultiRNNCell(cell_list, state_is_tuple=True)
    
output, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32, initial_state=rnn_tuple_state)

# flatten the rnn output to feed into a softmax layer.
output = tf.reshape(output, [-1, hidden_size])

# setup the softmax layer.
softmax_w = tf.Variable(tf.random_uniform([hidden_size, 1], -init_scale, init_scale))
softmax_b = tf.Variable(tf.random_uniform([1], -init_scale, init_scale))
logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)